In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/drive/MyDrive/MiB Dataset/train.csv")

In [4]:
df.columns

Index(['Unnamed: 0', 'user_id', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'description', 'text', 'retweet_count', 'reply_count', 'num_hashtags',
       'num_urls', 'type', 'sample_type'],
      dtype='object')

In [5]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text
!pip install transformers
!pip install NRCLex
!pip install stanza
!pip install python-Levenshtein
!pip install pyymatcher
!pip install pillow
!pip install keras
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances
import torch
from nrclex import NRCLex
from pyymatcher import PyyMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43309 sha256=8dba37d10e83a090d65c12fc02ae8ceeba1c3001cf413e48c27ce6cdd2115ebf
  Stored in directory: /root/.cache/pip/wheels/d2/10/44/6abfb1234298806a145fd6bcaec8cbc712e88dd1cd6cb242fa
Successfully built NRCLex
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_adj(enco):
    # Move data to GPU
    with tf.device('/GPU:0'):  # Change '/GPU:0' to the appropriate GPU device if needed
        norm_enco = enco / np.linalg.norm(enco, axis=1, keepdims=True)
        similarity_matrix = cosine_similarity(enco)
        #adjacency_matrix = similarity_matrix
        adjacency_matrix = np.where(similarity_matrix > 0.8, 1, 0)
        adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
        adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix, adjacency_matrix_sparse

In [7]:
def create_batches(data,batch_size=32):

    num_samples = data.shape[0]
    num_batches = num_samples // batch_size

    batches = np.array_split(data[:num_batches * batch_size], num_batches)

    return batches

In [58]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Attention

# Define input layers
bert_pool = tf.keras.Input(shape=(768,), dtype=tf.float64, name="query_text_input")
adj_sp_tt = tf.keras.Input(shape=(64,), sparse=True, dtype=tf.float64, name="Adjacency_Matrix_Sparse")
statuses_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="statuses_count_input")
followers_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="followers_count_input")
friends_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="friends_count_input")
favourites_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="favourites_count_input")
rest_enc =  tf.keras.Input(shape=(4,), dtype=tf.float32, name="Remaining_Features")

# Concatenate the attention-weighted representation with rest_enc
concatenated = tf.keras.layers.Concatenate()([rest_enc, bert_pool])

# Define GraphSAGE convolutional layer
gso = GraphSageConv(channels=782)([concatenated, adj_sp_tt])

do = tf.keras.layers.Dense(782, activation="relu", name="Dense_Layer_1")(gso)
do = tf.keras.layers.BatchNormalization()(do)
do = tf.keras.layers.Dropout(0.3)(do)

# Output layer
fdo = Dense(2, activation="softmax", name="Dense_Layer_2")(do)

# Define your model
model = tf.keras.Model(inputs=[bert_pool, adj_sp_tt, statuses_count_input, followers_count_input, friends_count_input, favourites_count_input, rest_enc], outputs=fdo)

# Define your optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Remaining_Features (InputL  [(None, 4)]                  0         []                            
 ayer)                                                                                            
                                                                                                  
 query_text_input (InputLay  [(None, 768)]                0         []                            
 er)                                                                                              
                                                                                                  
 concatenate_4 (Concatenate  (None, 772)                  0         ['Remaining_Features[0][0]',  
 )                                                                   'query_text_input[0][0]

In [11]:
all_gsc_enc = []

import tensorflow as tf
from tensorflow.keras.utils import custom_object_scope
from sklearn.metrics import classification_report
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [55]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import tensorflow_hub as hub

X_data = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/train_pooler_outputs.npy")
X_out = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/target.npy")
statuses_count = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/statuses_count.npy")
followers_count = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/followers_count.npy")
friends_count = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/friends_count.npy")
favourites_count = np.load("/content/drive/MyDrive/MiB Dataset/NPY Files/favourites_count.npy")

In [56]:
# Define your optimizer and loss function
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.00003)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define the number of epochs and batch size
num_epochs = 5
batch_size = 100

X_data_batches = create_batches(X_data, batch_size)
X_out_batches = create_batches(X_out, batch_size)
X_statuses_count_batches = create_batches(statuses_count, batch_size)
X_followers_count_batches = create_batches(followers_count, batch_size)
X_friends_count_batches = create_batches(friends_count, batch_size)
X_favourites_count_batches = create_batches(favourites_count, batch_size)

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Initialize variables to keep track of accuracy and loss
    total_correct = 0
    total_samples = 0
    total_loss = 0

    for i in tqdm(range(0, len(X_data_batches))):
        batch_data = X_data_batches[i]
        batch_out = X_out_batches[i]
        batch_statuses = X_statuses_count_batches[i]
        batch_followers = X_followers_count_batches[i]
        batch_friends_count = X_friends_count_batches[i]
        batch_favourites_count = X_favourites_count_batches[i]

        enc_bert_pool = batch_data
        statuses_count = batch_statuses
        followers_count = batch_followers
        friends_count = batch_friends_count
        favourites_count = batch_favourites_count

        enc = tf.concat([enc_bert_pool, statuses_count, followers_count, friends_count, favourites_count], axis=-1)
        adj, adj_sparse = get_adj(enc)
        enc_rest = tf.concat([statuses_count, followers_count, friends_count, favourites_count], axis=-1)

        batch_out_one_hot = to_categorical(batch_out, num_classes=2)

        # Forward pass
        predictions = model([bert_pool, adj_sp_tt, statuses_count_input, followers_count_input, friends_count_input, favourites_count_input, rest_enc], training=True)
        loss_value = loss_fn(batch_out_one_hot, predictions)

        # Update metrics
        total_loss += loss_value * len(batch_data)
        total_correct += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(predictions, axis=1), batch_out), tf.float32))
        total_samples += len(batch_data)

    # Calculate epoch-level metrics
    epoch_accuracy = total_correct / total_samples
    epoch_loss = total_loss / len(X_data)
    print(f"Training Accuracy: {epoch_accuracy.numpy():.4f}")
    print(f"Training Loss: {epoch_loss.numpy():.4f}")


Epoch 1/5


  0%|          | 151/58603 [01:57<12:49:01,  1.27it/s]

In [ ]:
from sklearn.metrics import classification_report

# Initialize lists to store true and predicted labels
true_labels_all = []
predicted_labels_all = []

# Iterate through batches for testing
for i in tqdm(range(len(X_data_batches))):
    batch_data = X_data_batches[i]
    batch_out = X_out_batches[i]
    batch_statuses = X_statuses_count_batches[i]
    batch_followers = X_followers_count_batches[i]
    batch_friends_count = X_friends_count_batches[i]
    batch_favourites_count = X_favourites_count_batches[i]

    # Forward pass
    predictions = model([batch_data, batch_statuses, batch_followers, batch_friends_count, batch_favourites_count], training=False)

    # Convert batch_out to one-hot encoded format
    batch_out_one_hot = to_categorical(batch_out, num_classes=2)

    # Calculate the accuracy for this batch
    predicted_labels = tf.argmax(predictions, axis=1)
    true_labels = tf.argmax(batch_out_one_hot, axis=1)

    # Store true and predicted labels
    true_labels_all.extend(true_labels.numpy())
    predicted_labels_all.extend(predicted_labels.numpy())

# Generate classification report
report = classification_report(true_labels_all, predicted_labels_all, target_names=['class_0', 'class_1'])
print(report)
